In [1]:
pip install datasets transformers


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install wikipedia-api


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Get wikipedia data (only technology and applied sciences categories)

In [3]:
import wikipediaapi
import pandas as pd
import os

In [ ]:
user_agent = "ananya (amantra@ncsu.edu)"

wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent=user_agent
)

def get_articles(category_name, level=0, max_level=2):
    if level > max_level:
        return set()

    cat = wiki_wiki.page(category_name)
    articles = set()

    for c in cat.categorymembers.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            articles = articles.union(get_articles(c.title, level=level+1, max_level=max_level))
        if c.ns == wikipediaapi.Namespace.MAIN:
            articles.add(c.title)

    return articles

def get_page_content(page_title):
    page = wiki_wiki.page(page_title)
    return page.text

categories = ["Category:Technology", "Category:Applied_sciences"]

articles = set()

for category in categories:
    titles = get_articles(category)
    for title in titles:
        content = get_page_content(title)
        articles.add((title, content))

if not os.path.exists('articles.csv'):
    df = pd.DataFrame(articles, columns=['Title', 'Content'])
    df.to_csv('articles.csv', index=False)

print(f"Total articles: {len(articles)}")
print(df.head())

In [18]:
!pip install gdown


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [22]:
import gdown

file_id = "1dy5ZRunNbLzrucIs27Om4XKcfCtjcUjX"
url = f"https://drive.google.com/uc?id={file_id}"
output = "articles.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv("articles.csv")
print(df.head())


Downloading...
From (original): https://drive.google.com/uc?id=1dy5ZRunNbLzrucIs27Om4XKcfCtjcUjX
From (redirected): https://drive.google.com/uc?id=1dy5ZRunNbLzrucIs27Om4XKcfCtjcUjX&confirm=t&uuid=f4cb242f-186b-40c3-b243-817df69a659c
To: /workspace/articles.csv
100%|██████████| 312M/312M [00:02<00:00, 124MB/s]  


                            Title  \
0                     Sideloading   
1                         Eternit   
2                AIM-9 Sidewinder   
3                           Swype   
4  Deindustrialisation by country   

                                             Content  
0  Sideloading describes the process of transferr...  
1  Eternit is a registered trademark for a brand ...  
2  The AIM-9 Sidewinder ("AIM" for "Air Intercept...  
3  Swype was a virtual keyboard for touchscreen s...  
4  Deindustrialisation refers to the process of s...  


In [33]:
import json
output_file = 'wiki_tech_applied_sci.jsonl'
with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        json_obj = {"text": row['Content']}
        f.write(json.dumps(json_obj) + '\n')

print(f"Total articles: {len(df)}")
print(f"Sample JSONL entry: {json.dumps(json.loads(open(output_file).readline()), indent=2)}")

Total articles: 41656
Sample JSONL entry: {
  "text": "Sideloading describes the process of transferring files between two local devices, in particular between a personal computer and a mobile device such as a mobile phone, smartphone, PDA, tablet, portable media player or e-reader.\nSideloading typically refers to media file transfer to a mobile device via USB, Bluetooth, WiFi or by writing to a memory card for insertion into the mobile device, but also applies to the transfer of apps from web sources that are not vendor-approved.\nWhen referring to Android apps, \"sideloading\" typically means installing an application package in APK format onto an Android device. Such packages are usually downloaded from websites other than the official app store Google Play. For Android users sideloading of apps is only possible if the user has allowed \"Unknown Sources\" in their Security Settings.\nWhen referring to iOS apps, \"sideloading\" means installing an app in IPA format onto an Apple dev

In [35]:
import json

input_file = "/workspace/wiki_tech_applied_sci.jsonl"
output_file = "/workspace/wiki_tech_applied_sci_fixed.jsonl"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        try:
            data = json.loads(line)
            if "text" in data and isinstance(data["text"], str):
                json.dump(data, outfile)
                outfile.write("\n")
        except json.JSONDecodeError:
            continue


In [4]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}
from datasets import load_dataset
datasets = load_dataset("json", data_files={"train": "/workspace/wiki_tech_applied_sci_fixed.jsonl"}, split="train")

In [5]:
datasets

Dataset({
    features: ['text'],
    num_rows: 41650
})

In [6]:
datasets["text"][35]

"The Mobile Telephone Switching Office (MTSO) is the mobile equivalent of a PSTN Central Office.  The MTSO contains the switching equipment or Mobile Switching Center (MSC) for routing mobile phone calls.  It also contains the equipment for controlling the cell sites that are connected to the MSC.\nThe systems in the MTSO are the heart of a cellular system.  It is responsible for interconnecting calls with the local and long distance landline telephone companies, compiling billing information (with the help of its CBM/SDM), etc. It also provides resources needed to efficiently serve a mobile subscriber such as registration, authentication, location updating and call routing.  Its subordinate BSC/RNC are responsible for assigning frequencies to each call, reassigning frequencies for handoffs, controlling handoffs so a mobile phone leaving one cell (formally known as BTS's) coverage area, can be switched automatically to a channel in the next cell.\nAll cellular systems have at least one

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    # df = pd.DataFrame(dataset[picks])
    selected_data = [dataset[i] for i in picks]
    df = pd.DataFrame(selected_data)
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [41]:
show_random_elements(datasets, 2)

## Causal Language Modeling
Causal language modeling is a good choice for text generation because it is autoregressive, learns to predict the next word in a sentence based on the words before it, just like how humans write naturally. This makes it ideal for generating coherent Wikipedia-style content. We dont need to capture birectional context, or require other methods like MLM that focus on filling in missing words. Since our goal is to create readable Wikipedia-like text, CLM helps model write in a logical and sequential manner.

In [8]:
model_checkpoint = "distilgpt2"

DistilGPT2 is an English-language model pre-trained with the supervision of the 124 M parameter version of GPT-2. DistilGPT2 (82 M parameters) was designed to be a faster, lighter version of GPT-2.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Our dataset does not have unique vocabulary, or in a non-standard language. Since we are using a pretrained model, Autoteknizer automatically picks the best tokenizer for the model. 

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"],truncation=True,max_length=1024,padding="max_length",return_tensors="pt")


In [11]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [12]:
# block_size = tokenizer.model_max_length
block_size = 128

In [13]:
# Concatenate all texts, split by chunks of max_len.
def group_texts(examples):
    
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_datasets.map(group_texts,batched=True,batch_size=1000,num_proc=4)
train_test_split = lm_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

In [15]:
tokenizer.decode(train_dataset[1]["input_ids"])

'\nIn the figure, the thickness of the propeller disc is assumed to be negligible. The boundary between the fluid in motion and fluid at rest is shown. Therefore, the flow is assumed to be taking place in an imaginary converging duct\n where:\n\nD = Diameter of the Propeller Disc.\nDs = Diameter at the Exit.\n\nIn the figure, across the propeller disc, velocities (C1 and C2) cannot change abruptly across the propeller disc as that will create a shockwave but the fan creates the pressure difference across the propeller disc.\n\n  \n '

In [16]:
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [18]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wiki",
    eval_strategy = "epoch",
    num_train_epochs =1,
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    weight_decay=0.01,
    save_steps=5000
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
# !pip install --upgrade -q wandb


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"
# import wandb
# wandb.init(mode="disabled")
# wandb.config

In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ananyamantravadi (ananyamantravadi-nc-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.368800,2.316501


TrainOutput(global_step=4686, training_loss=2.520073161692992, metrics={'train_runtime': 1529.9573, 'train_samples_per_second': 196.005, 'train_steps_per_second': 3.063, 'total_flos': 9794708695941120.0, 'train_loss': 2.520073161692992, 'epoch': 1.0})

In [ ]:
# model.save_pretrained("./wiki-trained")
# from google.colab import files
# files.download("/content/wiki-trained")

In [22]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 10.14


In [23]:
# Save fine-tuned model
trainer_filepath= "./epoch1"
trainer.model.save_pretrained(trainer_filepath)

In [24]:
input_text = "encryption methods in computer security"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# Set pad token explicitly
model.config.pad_token_id = model.config.eos_token_id

output = model.generate(
    input_ids,
    attention_mask=input_ids.ne(tokenizer.pad_token_id),
    max_length=100,         # Allow longer responses
    min_length=30,          # Ensure meaningful output
    temperature=0.7,        # Balance creativity & coherence
    top_p=0.9,              # Nucleus sampling for diverse yet relevant words
    repetition_penalty=1.2, # Reduce word repetition
    do_sample=True,         # Enable sampling for variety
    num_return_sequences=1  # Number of responses
)

# Decode and print output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


encryption methods in computer security.

Applications
In the past, it has been used to encrypt and decrypt data from a single device or network using various encryption techniques including:

A password-protected encrypted file (or any other type of hard disk) that can be accessed with either an administrator's keypad or by another user through access to its system on behalf of their computers. It is also possible for users who cannot afford to pay online tax credits at home via e-commerce platforms


In [25]:
input_text = "phishing is used to scam through"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# Set pad token explicitly
model.config.pad_token_id = model.config.eos_token_id

output = model.generate(
    input_ids,
    attention_mask=input_ids.ne(tokenizer.pad_token_id),
    max_length=100,         # Allow longer responses
    min_length=30,          # Ensure meaningful output
    temperature=0.7,        # Balance creativity & coherence
    top_p=0.9,              # Nucleus sampling for diverse yet relevant words
    repetition_penalty=1.2, # Reduce word repetition
    do_sample=True,         # Enable sampling for variety
    num_return_sequences=1  # Number of responses
)

# Decode and print output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


phishing is used to scam through a person's bank account for money that they can't legally use in order to buy, sell or distribute. The same method of fraudsters are often known as "virus" scams and are commonly referred to by the term "virus".
In some cases these types of fraudulent activities have been identified using their malware-like features such as spam filters, e-mails, mobile phone calls (or SMS), fake IDs etc., which were later found in real


In [28]:
input_text = "AI can be used in cybersecurity to automate"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

# Set pad token explicitly
model.config.pad_token_id = model.config.eos_token_id

output = model.generate(
    input_ids,
    attention_mask=input_ids.ne(tokenizer.pad_token_id),
    max_length=100,         # Allow longer responses
    min_length=30,          # Ensure meaningful output
    temperature=0.7,        # Balance creativity & coherence
    top_p=0.9,              # Nucleus sampling for diverse yet relevant words
    repetition_penalty=1.2, # Reduce word repetition
    do_sample=True,         # Enable sampling for variety
    num_return_sequences=1  # Number of responses
)

# Decode and print output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI can be used in cybersecurity to automate the execution of systems.
The first step is to implement a process that analyzes, and then provides data on, accesses or changes in information about, security, and other aspects such as user behavior. This analysis also includes methods for analyzing social networks including identity theft, malware targeting (MSO) attacks against users, criminal activity related activities, and network traffic control (NTP). An example of this method is by developing a database with an XML schema


## Load from saved model and continue training...

In [30]:
model2 = AutoModelForCausalLM.from_pretrained("./epoch1")

In [31]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wiki-epoch2",
    eval_strategy = "epoch",
    num_train_epochs =1,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_steps=5000
)

trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer2.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,2.319700,2.266976


TrainOutput(global_step=37485, training_loss=2.3458670577606044, metrics={'train_runtime': 1954.7968, 'train_samples_per_second': 153.407, 'train_steps_per_second': 19.176, 'total_flos': 9794708695941120.0, 'train_loss': 2.3458670577606044, 'epoch': 1.0})

In [32]:
eval_results2 = trainer2.evaluate()
print(f"Perplexity: {math.exp(eval_results2['eval_loss']):.2f}")

Perplexity: 9.65


In [33]:
# Save fine-tuned model
trainer_filepath= "./epoch2"
trainer.model.save_pretrained(trainer_filepath)

In [34]:
input_text = "AI can be used in cybersecurity to automate"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model2.device)

# Set pad token explicitly
model2.config.pad_token_id = model2.config.eos_token_id

output = model2.generate(
    input_ids,
    attention_mask=input_ids.ne(tokenizer.pad_token_id),
    max_length=100,         # Allow longer responses
    min_length=30,          # Ensure meaningful output
    temperature=0.7,        # Balance creativity & coherence
    top_p=0.9,              # Nucleus sampling for diverse yet relevant words
    repetition_penalty=1.2, # Reduce word repetition
    do_sample=True,         # Enable sampling for variety
    num_return_sequences=1  # Number of responses
)

# Decode and print output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI can be used in cybersecurity to automate security processes.
The CSA was developed by the CISA and is based on a concept from which it is based: "We're doing something that's easy, easy for people to do, and has the ability of all the people to do it." The idea behind this idea is not to build an automated system, but rather to provide the tools necessary under the circumstances to detect, analyze or prevent unauthorized activity. It would also be useful if there were
